The following code is to take a list of MP-IDs and get the gibbs energy of decomposition for each material. 

In [2]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry

from api_key import APIKEY

API_KEY = APIKEY

Take list of MP_IDs and get the material entries from the database

from entries, collect the elemental chemsys and composition and create lists with that data

create a new list with the material ids with "gga" added to search through the large list of data that will be generated in the next few cells

In [18]:
materials = ['mp-37153', 'mp-3414', 'mp-28159', 'mp-554379', 'mp-30159', 'mp-644828', 'mp-557856', 'mp-558168', 'mp-540958','mp-557090', 'mp-27743', 'mp-554129', 'mp-28153', 'mp-557519', 'mp-557426', 'mp-1323553', 'mp-7922', 'mp-29169', 'mp-557715', 'mp-13984', 'mp-542001', 'mp-556492', 'mp-639662', 'mp-1391848', 'mp-558211','mp-554737', 'mp-29842', 'mp-541347','mp-557432', 'mp-29170', 'mp-27988','mp-532810', 'mp-12442', 'mp-554835', 'mp-3817','mp-22439', 'mp-7921', 'mp-557668', 'mp-1102938', 'mp-505727', 'mp-556500', 'mp-726118','mp-7388', 'mp-29193', 'mp-14716', 'mp-558838', 'mp-757594', 'mp-28845','mp-20652', 'mp-341', 'mp-649616', 'mp-14715', 'mp-8943', 'mp-20458', 'mp-553981','mp-35311', 'mp-1391273', 'mp-13610', 'mp-13983', 'mp-8256', 'mp-561423', 'mp-643265', 'mp-554432', 'mp-14276', 'mp-23598', 'mp-572526', 'mp-541259', 'mp-560354', 'mp-560789', 'mp-29904', 'mp-8778', 'mp-29590','mp-555718', 'mp-31755', 'mp-556442', 'mp-30308', 'mp-17542', 'mp-27980', 'mp-759957', 'mp-1194388', 'mp-757234', 'mp-1402840', 'mp-17512']
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        material_entries = mpr.materials.search(material_ids=[mp_id])

        if material_entries:
            mp_entries.append(material_entries[0])

chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')]
materials_gga = [material + "-GGA" for material in materials]
materials_id = materials

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

the elemental list must not contain O (oxegen) or H (hydrogen) to compute the pourbaix data, the remove_o_and_h function removes those from the chemical composisiotn list

In [19]:
newlists = ['mp-37153', 'mp-3414', 'mp-28159', 'mp-554379', 'mp-30159', 'mp-644828', 'mp-557856', 'mp-558168', 'mp-540958','mp-557090', 'mp-27743', 'mp-554129', 'mp-28153', 'mp-557519', 'mp-557426', 'mp-1323553', 'mp-7922', 'mp-29169', 'mp-557715', 'mp-13984', 'mp-542001', 'mp-556492', 'mp-639662', 'mp-1391848', 'mp-558211','mp-554737', 'mp-29842', 'mp-541347','mp-557432', 'mp-29170', 'mp-27988','mp-532810', 'mp-12442', 'mp-554835', 'mp-3817','mp-22439', 'mp-7921', 'mp-557668', 'mp-1102938', 'mp-505727', 'mp-556500', 'mp-726118','mp-7388', 'mp-29193', 'mp-14716', 'mp-558838', 'mp-757594', 'mp-28845','mp-20652', 'mp-341', 'mp-649616', 'mp-14715', 'mp-8943', 'mp-20458', 'mp-553981','mp-35311', 'mp-1391273', 'mp-13610', 'mp-13983', 'mp-8256', 'mp-561423', 'mp-643265', 'mp-554432', 'mp-14276', 'mp-23598', 'mp-572526', 'mp-541259', 'mp-560354', 'mp-560789', 'mp-29904', 'mp-8778', 'mp-29590','mp-555718', 'mp-31755', 'mp-556442', 'mp-30308', 'mp-17542', 'mp-27980', 'mp-759957', 'mp-1194388', 'mp-757234', 'mp-1402840', 'mp-17512']

print(len(materials_id))
print(len(newlists))

83
83


In [20]:
def find_duplicates(input_list):
    duplicates = []
    seen = set()
    for item in input_list:
        if item in seen:
            duplicates.append(item)
        else:
            seen.add(item)
    return duplicates

my_list = [1, 2, 3, 2, 1, 5, 6, 5, 5, 7]
print(find_duplicates(newlists))

[]


In [21]:
from pymatgen.core.composition import Composition

def remove_o_and_h(composition):
    new_composition_elements = []
    for element, count in composition.get_el_amt_dict().items():
        if element not in ['O', 'H']:
            new_composition_elements.append(f"{element}{int(count)}")
    new_composition_str = ' '.join(new_composition_elements)
    return Composition(new_composition_str)

filtered_molecules = [remove_o_and_h(molecule) for molecule in composition_list]

the pourbaix charts also requires the elements to be in a percentage form, the calculate_percent_composition looks at the atoms per element in the composition list and converts it to a percentage

In [22]:
from pymatgen.core.composition import Composition

def calculate_percent_composition(molecule):
    total_atoms = sum(molecule.values())
    return {element.symbol: count / total_atoms for element, count in molecule.items()}

percent_compositions = [calculate_percent_composition(molecule) for molecule in filtered_molecules]

the following code takes the chemsys and percent composition and gets the pourbaix entries and converts it to data. some materials do not have a pourbaix entry to error is placed into the list, (lenght of list needs to match in a future function)

In [23]:
pbx_data = []
pbx_entries = []

with MPRester(API_KEY) as mpr:
    for i, x in zip(chemsys_list, percent_compositions):
        try:
            pourbaix_entries = mpr.get_pourbaix_entries(i)
            pbx_entries.append(pourbaix_entries)
            pbx_diagram = PourbaixDiagram(entries=pourbaix_entries, comp_dict=x)
            pbx_data.append(pbx_diagram)
        except Exception as e:
            pbx_data.append('error')

Retrieving ThermoDoc documents:   0%|          | 0/284 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/385 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/277 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/209 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/247 [00:00<?, ?it/s]

...

Retrieving ThermoDoc documents:   0%|          | 0/967 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/1049 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/967 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/377 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/249 [00:00<?, ?it/s]

the pourbaix entry responds with all viable pourbaix diagrams for a chemsys list, the following code searches through the list to get the correct diagram for the material of interest. 

In [24]:
matched_entries = []

for mat_id, entries in zip(materials_gga, pbx_entries):
    matched_entry = next((entry for entry in entries if entry.entry_id == mat_id), None)
    
    if matched_entry:
        matched_entries.append(matched_entry)
    else:
        matched_entries.append(None)

the stability data is then calculated at ph7 and 0V using the entry names and pbx_data

In [25]:
stabilities = []

for i, x in zip(pbx_data, matched_entries):
    try:
        stability = i.get_decomposition_energy(x, 7, 0)
        stabilities.append(stability)
    except Exception as e:
        stabilities.append("error") 


print(stabilities)


[0.5491808963884197, 0.42543007713483627, 0.9615071603069442, 'error', 1.4490996629451813, 'error', 1.0603941536346557, 1.3578262187495302, 'error', 0.9107493605782552, 0.9466168035516902, 1.5815281002310666, 1.3509582745553834, 'error', 'error', 'error', 0.7180260222751369, 0.9067619038298876, 1.3362281685921218, 0.7651058354690283, 0.2579253860172863, 'error', 'error', 'error', 'error', 0.948417299305657, 0.7848489643785639, 0.8826663639401474, 1.0217634596047367, 1.0345589531329005, 1.5313921582013004, 'error', 1.1616193255467109, 'error', 0.8623835672815053, 'error', 0.7558275780707273, 'error', 'error', 1.5451010279265625, 1.8619025604332542, 2.622185165952526, 0.7955194750511478, 0.07944177418243836, 0.8810600308151679, 'error', 0.6908198947742511, 0.42558883681275805, 0.546332571355435, 0.8787412109220026, 'error', 0.8606883090293819, 1.1699781783506182, 0.6889458847483212, 'error', 'error', 'error', 0.6096687028844632, 0.8146456006523461, 0.7619169356523456, 0.16676177491096889

In [26]:
combined_list = list(zip(materials_id, stabilities))
data_without_error = [item for item in combined_list if item[1] != 'error']
filtered_data = [item for item in data_without_error if float(item[1]) <= 0.5]
mp_identifiers = [item[0] for item in filtered_data]

In [27]:
print("total materials:",len(materials_id))
print("materials without error:",len(data_without_error))
print("stable materials:",len(filtered_data))
print(mp_identifiers)

total materials: 83
materials without error: 55
stable materials: 9
['mp-3414', 'mp-542001', 'mp-29193', 'mp-28845', 'mp-561423', 'mp-14276', 'mp-29590', 'mp-556442', 'mp-27980']
